# 경기 결과를 기록하면 이를 CSV 파일로 변환시킨다.

## 이후 더 좋은 방법이 있다면 자동화를 할 수 있도록 한다.

In [270]:
# 모듈 import

from bs4 import BeautifulSoup
import re

from selenium import webdriver
from datetime import datetime

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

import folium

In [271]:
# webdriver options
options = webdriver.ChromeOptions()

options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [272]:
def select_inning(inning_index): # 이닝 버튼 클릭하는 함수
    try:
        # 모든 이닝 버튼 가져오기
        inning_buttons = wd.find_elements(By.CLASS_NAME, 'SetTab_button_tab___H3-s')

        # 원하는 이닝 클릭
        inning_buttons[inning_index].click()

        # aria-selected 속성이 true로 바뀔 때까지 대기
        time.sleep(1)
        selected = inning_buttons[inning_index].get_attribute("aria-selected")
        while selected != "true":
            time.sleep(1)
            selected = inning_buttons[inning_index].get_attribute("aria-selected")

        return 1

    except IndexError:
        return 0

    except Exception as e:
        print(f"An error occurred: {e}")
        return -1

In [273]:
# 이닝 데이터 가져오는 함수
def get_inning_data():
    try:
        # 이닝 데이터 찾기 (여기서는 클래스 이름 예시)
        # play_by_play = wd.find_elements(By.CLASS_NAME, 'RelayInfo_relay_info_group__1aqtH')  # 요소의 클래스 이름 필요
        play_by_play = wd.find_elements(By.CSS_SELECTOR, '[class ^= "RelayInfo_relay_info_group"]')
        for play in play_by_play:
            soupInning = BeautifulSoup(play.get_attribute('innerHTML'), 'html.parser')
            return soupInning
    except Exception as e:
        print(f"Error fetching inning data: {e}")

In [274]:
# HTML을 한 줄씩 읽어들여서 태그로 시작하지 않는 줄만 return
def non_tag_lines(html_string):

    result = []
    
    # HTML을 줄 단위로 분리
    lines = html_string.splitlines()
    
    # 태그로 시작하는지 여부를 판단하는 정규 표현식
    tag_pattern = re.compile(r'^\s*<[^>]+>')

    for line in lines:
        # 태그로 시작하지 않는 줄만 출력
        if not tag_pattern.match(line):
            result.append(line.strip())  # strip()으로 양 끝의 공백 제거
    
    return result

In [275]:
def make_raw_data(text) :

    raw_data = []
    turn_data = []
    start_found = False

    # 이닝 소개를 찾기 위한 패턴
    pattern_inning = re.compile(r'\d+회\w - \w+공격')

    # 한 턴을 찾기 위한 패턴 (턴 : 한 타석)
    start_marker = "투구 위치 보기"
    end_marker_pattern = re.compile(r'\w+ 선수 페이지')

    for line in text :
        if start_found == False and pattern_inning.match(line) :
            raw_data.append(line)
        elif start_marker in line :
            start_found = True
            continue
        elif start_found == True and pattern_inning.match(line) :
            raw_data.append(turn_data)
            turn_data = []
            start_fount = False
            raw_data.append(line)
        elif end_marker_pattern.match(line):
            if start_found :
                raw_data.append(turn_data)
                turn_data = []
                start_found = False
        elif start_found :
            turn_data.append(line.strip())
    
    raw_data.append(turn_data)
    turn_data = []
    
    return raw_data



In [276]:
def raw_to_list(raw_data) :
    
    cache = []
    edited_data = []

    for data in raw_data :
        if isinstance(data, str):
            edited_data.append(data)
            continue
        elif isinstance(data, list):
            index = 0
            while(index < len(data)) :
                if data[index] == "기록 펼치기" :
                    del data[index : index + 4]
                    continue
                elif data[index] == "볼카운트" :
                    del data[index : index + 2]
                    continue
                elif data[index] == "구" :
                    cache.append(data[index - 1])
                    cache.append(data[index])
                    cache.append(data[index + 1])
                    cache.append(data[index + 2])
                    cache.append(data[index + 3])
                    item_ = [cache[0] + cache[1], cache[2], cache[3], cache[4]]
                    del data[index - 1 : index + 4]
                    index -= 1
                    edited_data.pop()
                    edited_data.append(item_)
                    cache = []
                    continue
                elif data[index] == "교체" :
                    for i in range(5) :
                        cache.append(data[index + i])
                    del data[index : index + 5]
                    edited_data.append(cache)
                else :
                    edited_data.append(data[index])
                    index += 1
                    
    return edited_data

In [277]:
def split_inning(data):
    # 이닝 소개를 찾기 위한 패턴
    pattern_inning = re.compile(r'\d+회\w - \w+공격')

    segments = []
    start_index = 0

    for i in range(len(data)):
        if isinstance(data[i], str) and pattern_inning.match(data[i]):
            segments.append(data[start_index:i])
            start_index = i

    # Add the last segment if there's anything left after the final target element
    if start_index < len(data):
        segments.append(data[start_index:])

    for segment in segments :
        if len(segments) > 2 and segment == [] :
            segments.remove(segment)

    segments[0][1:] = segments[0][1:][::-1]
    segments[1][1:] = segments[1][1:][::-1]

    return segments[0], segments[1]

# Get Event

In [278]:
# Event.json 생성

import json
from collections import OrderedDict

def get_event(soup) :
    formatted_soup = soup.prettify()
    lines = non_tag_lines(formatted_soup) # 경기 내용만 추출
    raw = make_raw_data(lines) # 초기 데이터 생성
    list_data = raw_to_list(raw) # 데이터 1차 정제
    bottom, top = split_inning(list_data)# 데이터 2차 정제

    return bottom, top

In [279]:
# URL 구성하기

base_URL = "https://m.sports.naver.com/game/"
date = "20240908"
# date = datetime.today().strftime("%Y%m%d")

team = input("팀 이름")
foot_URL = "02024/relay"

URL = base_URL + date + team + foot_URL

URL

'https://m.sports.naver.com/game/20240908HHLG02024/relay'

# Main Function

In [281]:
# URL 접속 후 데이터 추출하기
wd = webdriver.Chrome(service = Service(), options=options)
            
wd.get(URL)
time.sleep(1)  #웹페이지 연결할 동안 1초 대기

# 이닝 순서에 따라 클릭하고 데이터 수집
for inning_index in range(0, 13): # 최대 12회까지 진행
    flag = 0
    flag = select_inning(inning_index) # 이닝을 바꾸는 함수
    if flag == 1 :
        soupInning = get_inning_data() # 이닝 데이터 가져오는 함수
        data_bottom, data_top = get_event(soupInning)
        print(data_top)
        print(data_bottom)
# 브라우저 종료
wd.quit()


['1회초 - 한화공격', ['1구', '볼', '143Km/h', '직구'], ['2구', '타격', '144Km/h', '직구'], '황영묵 : 좌익수 플라이 아웃', ['1구', '파울', '144Km/h', '직구'], ['2구', '타격', '145Km/h', '직구'], '장진혁 : 2루수 라인드라이브 아웃', ['1구', '볼', '149Km/h', '직구'], ['2구', '볼', '131Km/h', '체인지업'], ['3구', '볼', '144Km/h', '직구'], ['4구', '스트라이크', '145Km/h', '직구'], ['5구', '볼', '145Km/h', '직구'], '페라자 : 볼넷', ['1구', '볼', '144Km/h', '투심'], ['2구', '헛스윙', '145Km/h', '직구'], ['3구', '타격', '147Km/h', '직구'], '노시환 : 3루수 앞 땅볼로 출루', '1루주자 페라자 : 포스아웃 (3루수-&gt;2루수 2루 터치아웃)']
['1회말 - LG공격', ['1구', '볼', '140Km/h', '직구'], ['2구', '볼', '139Km/h', '직구'], ['3구', '스트라이크', '140Km/h', '직구'], ['4구', '타격', '140Km/h', '직구'], '홍창기 : 좌중간 2루타', ['1구', '볼', '113Km/h', '커브'], ['2구', '스트라이크', '120Km/h', '체인지업'], ['3구', '타격', '140Km/h', '직구'], '김현수 : 우익수 앞 1루타', '2루주자 홍창기 : 3루까지 진루', ['1구', '볼', '120Km/h', '체인지업'], ['2구', '스트라이크', '120Km/h', '체인지업'], ['3구', '스트라이크', '115Km/h', '커브'], ['4구', '스트라이크', '142Km/h', '직구'], '오스틴 : 삼진 아웃', ['1구', '볼', '116Km/h', '커브'], ['2구', '파울', '123Km